In [235]:
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from sklearn.feature_extraction.text import CountVectorizer
#packages for data cleaning
import re
import string

In [132]:
df = pd.read_csv('./amz_tshirt_reviews.csv')

In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40521 entries, 0 to 40520
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_name  40521 non-null  object 
 1   attr          0 non-null      float64
 2   scores        40441 non-null  object 
 3   review_title  40441 non-null  object 
 4   reviews       40521 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.5+ MB


#### when I webscrapped the data, I had a bug that caused the 'attr' column to be null, I need to re-run the webscrapping ipynb file and get the complete dataset 

In [37]:
df.shape

(40521, 5)

In [186]:
df.sample(10)

,product_name,attr,scores,review_title,reviews
32624,Hanes Men's Tagless Cotton V-Neck Undershirt – Multiple Packs and Colors,NaN,5,comfort,got these also for my father. he likes the v neck doesn't like anything tight up by his neck.
8902,Under Armour Men's Tech 2.0 Short-Sleeve T-Shirt,NaN,4,fit is good,bought to replace an older under armor shirt. the new one fits fine but the material seems thinner and not as good as the older ones i have.
1550,"Gildan Men's Crew T-Shirts, Multipack",NaN,2,i liked white t-shirts very much (gives nice soft feel) and ...,"all shows in comparison. when i bought banana republic soft tee (yes, it is $20), i realized that i was wasting money on guildan t-shirts. day and night! now i cannot even put on guildan t-shirts and will be disposing of them, since i see no use for the product. after using banana republic for few days, guildan t-shirts seem harsh and deformed, not soft on skin, heavy, and very uncomfortable comparable to banana republic soft tees. the difference is obviously in much much quality of fabri..."
38396,"Comfort Colors Men's Adult Short Sleeve Tee, Style 1717",NaN,1,color is not even close to what is shown.,color of the neon red orange isn't even close to what's shown. looks more like coral/pink/peach color.
17290,Amazon Essentials Women's 2-Pack Classic-Fit Short-Sleeve Crewneck T-Shirt,NaN,5,my new favorite,"i like to have a few basic tees. i usually get them from old navy, but for the same price i think these are way nicer. the fabric has a nice feel and the fit is just about prefect. i have the xxl and they manage the perfect mix of still having some breathing room without being shapeless. i went and bought two more packs in different colors!"
23687,"Champion Men's Classic T-Shirt, Screen Print Script",NaN,5,champion t-shirt,the shirt fits well and looks good. i had to send two back because they didn't fit as well as the first one i ordered. i ordered another one and it fits well.
2067,Hanes Men's Essentials Short Sleeve T-shirt Value Pack (4-pack),NaN,3,not as i expected. okay for a cheap t-shirt for work/working out.,"i bought xl size expecting to be able to wash and dry them and have them shrink quite a bit, and get them to what a large is before washing/drying, because the large size i got from walmart shrunk down to a medium. well, these are way bigger than i expected, and the fit is different.it's funny because most people in the reviews are complaining about the shrink, when that's what i was looking for. if you need an xl and expect a big shirt, these are definitely big. they'll shrink, but not..."
16220,Hanes Men’s Short Sleeve Graphic T-shirt Collection,NaN,5,with all the cheap junk tshirts out there..this one is old school good,old fashioned good quality t shirt.. the one last last year's before falling apart.
20949,5 Pack: Men’s Dry-Fit Moisture Wicking Active Athletic Performance Crew T-Shirt,NaN,3,camisa deportiva,camisa de buena calidad “” es buen material pero las camisas blancas tiene un problema y es que se ve tú cuerpo no es cubridora tengo tatuajes y se ven a la distancia
17420,Amazon Essentials Women's 2-Pack Classic-Fit Short-Sleeve Crewneck T-Shirt,NaN,2,admit i should have known better than to order something with so much spandex,"...not the droid you're looking for... runs about 2 sizes too small. very trendy but not an everyday, hide-every-flaw, loose-fitting, basic black t-shirt. must be worn under something else. admit i should have known better than to order something with so much spandex. live and learn."


In [155]:
df.review_title.value_counts()

\nComfortable\n                            156
\nPerfect\n                                149
\nGreat shirt\n                            140
\nRuns small\n                             134
\nGood quality\n                           133
                                          ... 
\nPrice was right for around home.\n         1
\nNot what you think you are getting.\n      1
\nSoft, fit well\n                           1
\nHe love his T shirt\n                      1
\nPoor variety\n                             1
Name: review_title, Length: 16147, dtype: int64

In [177]:
#increase the column width to be able to view long reviews 
pd.set_option('max_colwidth', 500)
df['reviews'].sample(2)

17750                          for the money, i can't imagine a better value. these are super soft, comfy and flattering. the fabric isn't too thin or too heavy -- just right. these tees also have just the right amount of stretch/cling -- not too clingy but not at all loose/baggy.  i just got them so i'm not sure how well they'll hold up, but even if it's only for a little while, these are worth it.
7214       these shirts are great. there are many reviews stating that they are not heavyweight but they are much heavier than the regular ones. also, i would order one size larger than normal because they do shrink in the wash. i purchased the white and they are very bright and not faded at all and unlike many t-shirt’s, they are not too narrow but just the right width. definitely worth purchasing.
Name: reviews, dtype: object

## clean 'reviews' column 

In [153]:
def clean_reviews(text):
    # make it all lowercase 
    text = text.lower()
    # remove text between </>
    text = re.sub('\<[^>]*\>', '', text)
    # remove punctuation 
    #--- I am not going to remove punctuations for now because i might split the sentences. 
    # text = re.sub('[%s]' % re.escape(string.punctuation), '', text) 

    # remove all '\n\n'
    text = re.sub(r"\n\n ", ' ', text)
    text = text.rstrip("\n")
    
    return text
df.reviews = df.reviews.apply(clean_reviews)

## clean 'review_title'

In [174]:
def clean_titles(title):
    title = str(title)
    #make all lowercase 
    title = title.lower()
    # remove all '\n'
    title = re.sub(r"\n", '', title)
    return title

df.review_title = df.review_title.apply(clean_titles) 

## Extract review score as int

In [185]:
df.scores.fillna('9')
df.scores = df.scores.str.slice(start = 0, stop =1)

## split the reviews columns

In [220]:
df['reviews_splitted'] = df.reviews.apply(lambda x: re.split('[,.]', str(x)))

In [221]:
df.reviews_splitted

0        [  i recommend these t-shirts,  i am 5'8" and 140",  i do not like tight t-shirts so i bought a large,  i am happy with it,  although i am sure that a medium would have worked, i will compare the gilden t-shirt with the fruit of the loom platinum that you will see at wal-mart, pro - the gilden appears to be better built and the sizing was consistent on the two t-shirts that came in the package,  the neck was smaller,  without being tight,  i am guessing that fabric around the neck will not w...
1        [  my husband actually has t-shirts older than our college-age grandson,   some looked as if he was present and against the wall at the valentine's day massacre,   i've slowly been trashing them as he brings them to be laundered and i was looking to replace some of his around-the house t-shirts,   he likes them reasonably soft and medium weight,   i sent off for a set of two of these and he really likes them,   he wore them one right after the other,  and the second they had we

In [222]:
df['review_length'] = df.reviews_splitted.apply(lambda x: len(x))

## CountVectorize

In [237]:
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(df.reviews)
data_dtm = pd.DataFrame(data_cv.toarray()
                        , columns = cv.get_feature_names_out())
data_dtm.index = data.index
data_dtm

In [193]:
df.review_length

0        1
1        1
2        1
3        1
4        1
        ..
40516    1
40517    1
40518    1
40519    1
40520    1
Name: review_length, Length: 40521, dtype: int64

In [150]:
sentence = 'Hello There my name is hend. What is your name?'
toekns = nltk.word_tokenize(sentence)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/home/vscode/nltk_data'
    - '/opt/conda/nltk_data'
    - '/opt/conda/share/nltk_data'
    - '/opt/conda/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [149]:
df['tokenized_reviews'] = df['reviews'].apply(word_tokenize)
#I must read 

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/home/vscode/nltk_data'
    - '/opt/conda/nltk_data'
    - '/opt/conda/share/nltk_data'
    - '/opt/conda/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [7]:
df.sample(5)

,product_name,attr,scores,review_title,reviews
27843,Hanes Ultimate Men's 5-Pack Tagless ComfortSof...,NaN,5.0 out of 5 stars,\nBest value undershirt!\n,"<span class=""a-size-base review-text review-te..."
4559,Fruit of the Loom Men's Stay Tucked Crew T-Shirt,NaN,5.0 out of 5 stars,\nFinally a shirt that doesn't shrink to the p...,"<span class=""a-size-base review-text review-te..."
12748,Hanes Men's Beefy Heavyweight Short Sleeve T-S...,NaN,2.0 out of 5 stars,\nOdd fit between neck and right shoulder.\n,"<span class=""a-size-base review-text review-te..."
18668,Amazon Essentials Women's 2-Pack V-Neck Classi...,NaN,5.0 out of 5 stars,\nSo nice!\n,"<span class=""a-size-base review-text review-te..."
28981,"Gildan Men's DryBlend T-Shirt, Style G8000, 2-...",NaN,5.0 out of 5 stars,\nGood shirts. Great value. Very happy\n,"<span class=""a-size-base review-text review-te..."
